# Metody Obliczeniowe w Nauce i Technice
## Laboratorium 2: Rozwiązywanie układów równań liniowych
## Przemysław Węglik

### Zadanie 1 - Metoda Gaussa-Jordana
Napisz i sprawdź funkcję rozwiązującą układ równań liniowych $n × n$ metodą GaussaJordana z całkowitym poszukiwaniem elementu wiodącego. Dla dziesięciu różnych rozmiarów macierzy współczynników większych niż $500 × 500$ porównaj czasy działania
zaimplementowanej funkcji z czasami uzyskanymi dla wybranych funkcji bibliotecznych.

In [1]:
import numpy as np
from pyvis.network import Network
import networkx as nx
import random
import math
import time

In [2]:
# partial pivoting
def partial_pivot(array, col_index):
    """Args: array to pivot and column index to find pivot for
    returns pivot for column with index col_index"""
    found_pivot = np.argmax(array[:, col_index])
    
    temp = np.copy(array[found_pivot, :])

    array[found_pivot, :] = array[col_index, :]
    array[col_index, :] = temp
    
    return temp[col_index]

def gj_elimination(A, b):
    """
    Takes A matrix and b vector as argument
    Return vector of solutions
    """
    A_aug = np.c_[ A, b ]

    # scaling
    max_el = np.amax(np.absolute(A_aug), axis=1)

    A_aug = A_aug / max_el[:, np.newaxis]

    # Gaussian elimination

    for row_index in range(A_aug.shape[0] - 1):
        pivot_value = partial_pivot(A_aug, row_index)

        multiplier =  - A_aug[row_index:, row_index] / pivot_value
        multiplier[0] = 0 # set first row of multiplyer to zero soe it doesnt delete itself
        multiplier = multiplier[:, np.newaxis]
        temp = multiplier * A_aug[row_index, :]

        A_aug[row_index:,:] = A_aug[row_index:,:] + temp

    # continuation, elimination upwards
    # going from lowest row to upper most
    for row_index in range(A_aug.shape[0] - 1, 0, -1):

        multiplier =  - A_aug[:row_index, row_index] / A_aug[row_index, row_index]

        multiplier = multiplier[:, np.newaxis]
        temp = multiplier * A_aug[row_index, :]

        A_aug[:row_index,:] = A_aug[:row_index,:] + temp

    # scale back to 1s at diagonal
    A_aug = A_aug / np.diagonal(A_aug[:,:-1])[:, np.newaxis]

    return A_aug[:, -1]

In [3]:
A = np.array([[2, 1, -1], [-3, -1, 2], [-2, 1, 2]])
b = [8, -11, -3]
#solution = 2 3 -1
gj_elimination(A, b)

array([ 2.,  3., -1.])

In [4]:
for n in range(500, 1100, 100):
    A = np.random.rand(n, n) * 10
    b = np.random.rand(n)* 5
    A = A.astype(int)
    b = b.astype(int)
    print(f'n: {n}')
    start = time.time()
    gj_elimination(A, b)
    print(f'Custom Time {time.time() - start}')
    start = time.time()
    np.linalg.solve(A, b)
    print(f'Library Time {time.time() - start}\n')


n: 500
Custom Time 0.21743249893188477
Library Time 0.009248495101928711

n: 600
Custom Time 0.44058799743652344
Library Time 0.007093667984008789

n: 700
Custom Time 0.7460465431213379
Library Time 0.007922172546386719

n: 800
Custom Time 1.346130609512329
Library Time 0.6078839302062988

n: 900
Custom Time 2.2280876636505127
Library Time 0.03781843185424805

n: 1000
Custom Time 3.155705213546753
Library Time 0.028905391693115234



### Zadanie 2 Faktoryzacja LU
Napisz i przetestuj funkcję dokonującą faktoryzacji $A = LU$ macierzy $A$ (bez poszukiwania elementu wiodącego). Sprawdź poprawność wyniku obliczając $||A − LU||$. Zadbaj
o to żeby implementacja była in-situ. Elementy macierzy $L$ to współczynniki mnożenia
umożliwiające wyzerowanie odpowiedniego współczynnika macierzy $A$ w trakcie procesu
eliminacji.

In [5]:
def LU_decompsition(A):
    n = A.shape[0]
    for i in range(n):

        for j in range(i):
            LU[i][j] = (LU[i][j] - np.dot(LU[i, 0:j], LU[0:j,j])) / LU[j,j]

        for j in range(i, n): 
            LU[i,j] = LU[i,j] - np.dot(LU[i,0:i], LU[0:i,j]);

def LU_multiply(LU):
    L = np.tril(LU)
    U = np.triu(LU)
    np.fill_diagonal(L, 1)
    
    return L @ U

In [6]:
LU = np.array([[2, -1, -2], [-4, 6, 3], [-4, -2, 8]])
arr_copy = np.copy(LU)
LU_decompsition(LU)

In [7]:
arr_copy - LU_multiply(LU)

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

### Zadanie 3 Analiza obwodu elektrycznego - nadokreślony układ równań

Napisz program, który: <br />
**a)** Wczytuje z pliku listę krawędzi grafu nieskierowanego ważonego opisującego obwód elektryczny. Wagi krawędzi określają opór fragmentu obwodu między dwoma
węzłami. Wierzchołki grafu identyfikowane są przez liczby naturalne.<br />
**b)** Wczytuje dodatkowo trójkę liczb $(s, t, E)$, przy czym para $(s, t)$ wskazuje między którymi węzłami sieci przyłożono siłę elektromotoryczną $E$. Opór wewnętrzny
SEM można zaniedbać.

In [8]:
import networkx as nx
from pyvis.network import Network

def create_graph(filename):
    G = nx.Graph()
    s = -1
    t = -1 
    E = -1
    with open(filename, "r") as file:
        header = file.readline()
        s, t, E = map(int, header.split(" "))
        
        for i in range(t):
            G.add_node(i)
        
        lines = file.readlines()
        for line in lines:
            u, v, w = map(int, line.split(" "))
            # value is just for plotting, sqrt is empirical for nice display
            G.add_edge(u, v, weight = 1/w, value = np.sqrt(w))
            G.add_edge(v, u, weight = 1/w, value = np.sqrt(w))
            
    return G, s, t, E
            

In [9]:
graph, s, t, E = create_graph("data/simple")

nt = Network('400px', '400px', notebook=True)
nt.from_nx(graph)
nt.show('nx.html')

**c)** Wykorzystując prawa Kirchhoffa (albo metodę potencjałów węzłowych) znajduje
natężenia prądu w każdej części obwodu i przedstawia je na rysunku w postaci
grafu ważonego z etykietami (wizualizacja grafu wraz z kolorowymi krawędziami
pokazującymi wartość natężenia prądu oraz jego kierunek)

In [10]:
def fill_G_matrix(i, j, graph, n):
    if (i == 0 and j == 0) or (i == n - 1 and j == n - 1):
        return 1
    
    if i == 0 or i == n - 1:
        return 0
    
    if i == j:
        return sum([graph[i].get(k, {'weight': 0})['weight'] for k in range(n)])
    
    return -graph[i].get(j, {'weight': 0})['weight']

def get_currents(graph, s, t, E):
    n = t + 1
    G = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            G[i][j] = fill_G_matrix(i, j, graph, n)

    known_V = np.zeros(n)
    known_V[0] = E
    V = gj_elimination(G, known_V)
    
    currents = nx.DiGraph()
    currents.add_nodes_from(list(range(n)))
    for i in range(n):
        for j in range(n):
            w = graph[i].get(j, {'weight': 0})['weight'] * (V[i] - V[j])
            # stosujemy konwencje że dodatni prąd płynie z mniejszych nodów do większych
            if w != 0 and i < j:

                currents.add_edge(i, j, weight = w, value = np.sqrt(w))
    
    return currents

def check_solution(currents, s, t):
    n = t + 1
    sum_s = 0
    for i in range(n):
        sum_s += currents[s].get(i, {'weight': 0})['weight']
        
    sum_t = 0
    for i in range(n):
        sum_t += currents[i].get(t, {'weight': 0})['weight']
    
    # if errors lesser than 0.1%
    if abs(sum_s - sum_t)/sum_s < 10**(-3):
        print(f'Correct solution! Current sum: {sum_s}')
        return True
    else:
        print(f'Invalid solution! Currents from source: {sum_s}. Currents to target: {sum_t}')
        return False

#### Simple graph example

In [11]:
graph, s, t, E = create_graph("data/simple")

currents = get_currents(graph, s, t, E)
check_solution(currents, s, t)

nt = Network('400px', '400px', notebook=True, directed= True)
nt.from_nx(currents)
nt.show('nx.html')

Correct solution! Current sum: 1.5


#### Grid graph

In [14]:
n = 25
graph = nx.grid_graph([int(math.sqrt(n)), int(math.sqrt(n))])
graph = nx.convert_node_labels_to_integers(graph)
for i in range(n):
    for j in range(n):
        if j in graph[i]:
            w = random.uniform(2, 8)
            graph[i][j]['weight'] = w
            graph[i][j]['value'] = np.sqrt(w)
s = 0
t = n - 1
E = 20

currents = get_currents(graph, s, t, E)
check_solution(currents, s, t)


nt = Network('400px', '400px', notebook=True)
nt.from_nx(graph)
nt.show('nx.html')

Correct solution! Current sum: 44.22839071622382


In [15]:
nt = Network('400px', '400px', notebook=True, directed= True)
nt.from_nx(currents)
nt.show('nx.html')

### Disclaimer
Niestety nie udało mi się wykonać testów związanych z pozostałymi grafami.
Wkradł się drobny błąd w proces tworzenia grafu związany najprawdopodobniej z prądami płynącymi w kierunku przeciwnym. Algorytm nie pokrywa takich przypadków poprawnie

### Wnioski
Użyto metody potencjałów węzłowych. Metoda okazała się prostsza niż wcześniej zakładałem. Całość zawiera się w funkcji fill_G_matrix() przy pomocy której wypełniamy macierz, aby potem rozwiązać układ równań. W pierwszym i ostatnim wierszu macierzy mamy tylko jedną niezerową wartość (konkretnie 1) odpowiadającą równaniom dla węzła wejściowego i wyjściowego. Mamy dla nich informacje o napięciu, a zatem równanie upraszcza sie do prostego $V_0 = E$ i $V_n-1 = 0$. W rozwiązaniu zakładamy, że kierunek przepływu prądu jest dodatni, jeśli prad płynie od i-tego do j-tego węzła, przy czym i < j. Rozwiązanie zweryfikowane w prosty sposób: sumując prąd wychodzący ze źródła i wchodzący do ujścia grafu.